In [1]:
!pip3 install tabula-py numpy pandas pyPDF2 requests beautifulsoup4 tqdm

In [2]:
import tabula
import numpy as np
import pandas as pd
import os 
from pathlib import Path
import PyPDF2
import re
import requests
import json
import time
import requests
from bs4 import BeautifulSoup as bs4
import urllib
from multiprocessing import Pool,Manager,Queue
from tqdm import tqdm

In [3]:
req=requests.get("http://ggsipu.ac.in/ExamResults/ExamResultsmain.htm")

bs=bs4(req.text)

ahrefs=[i.attrs['href'] for i in bs.table.find_all("a")]

In [4]:
try:
    os.mkdir("files")
except FileExistsError:
    pass
os.chdir("files")

In [5]:
for i in range(len(ahrefs)):
    ahrefs[i]='http://ggsipu.ac.in/ExamResults/'+urllib.parse.unquote(ahrefs[i])
ahrefs

['http://ggsipu.ac.in/ExamResults/2020/291020/8th SEM_2015_015_B.TECH(IT)_DECLARE_RESULT_MAY2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/291020/8th SEM_2015_032_B.TECH(CSE)_DECLARE_RESULT_MAY2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/291020/8th SEM_2015_128_B.TECH(ECE)_DECLARE_RESULT_MAY2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/261020/MA (English) 4th Semester Result September 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/261020/MA(Economics) 4th Semestr Result September 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/005_M.Tech_SP_Result_May 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/006_M.Tech_RF_Result_May 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/007_M.Tech_DC_Result_May 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/008_M.Tech_IS_Result_May 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/MBA 2nd Semester Result End Term June 2020.pdf',
 'http://ggsipu.ac.in/ExamResults/2020/201020/MBA(Weekend) 2nd Sem

In [6]:
# download(ahrefs[1],"1.pdf")

In [7]:
filenames=os.listdir()
# filenames = [
#              os.path.expanduser('/home/parth/Documents/USICT/it_res.pdf'),
#              os.path.expanduser('/home/parth/Documents/USICT/cse_res.pdf'),
#              os.path.expanduser('/home/parth/Documents/USICT/ece_res.pdf')]
# filenames = [
#     os.path.expanduser('~/Documents/USICT/ipu_results/cse_even_sems.pdf'),
#     os.path.expanduser('~/Documents/USICT/ipu_results/ece_even_sems.pdf')
# ]

In [8]:
scheme_reg = re.compile(r'scheme\s+of\s+examinations',re.IGNORECASE)
institution_reg = re.compile(r'institution\s*:\s*([\w\n(,)\-.& ]+)S\.No',re.IGNORECASE)
sem_reg = re.compile(r'se\s?m[.//\w\n]+:\s+([\w\n]+)',re.IGNORECASE)
programme_reg = re.compile(r'programme\s+name:\s+([\w(,)\-& \n]+)SchemeID',re.IGNORECASE)
branch_reg = re.compile(r'[\w ,\-&]+\(([\w, \-]+)\)')

In [9]:
def filter_text(text):
    return ' '.join(text.strip().split()).replace('&','and').replace('.','').title()
filter_text('Dr. Akhilesh Das Gupta Institute Of Technology And Management(Formerly Niec)')

'Dr Akhilesh Das Gupta Institute Of Technology And Management(Formerly Niec)'

In [10]:
def get_info(text) :
    college = filter_text(institution_reg.search(text)[1])
    semester = int(sem_reg.search(text)[1].strip())
    course = filter_text(programme_reg.search(text)[1])
    try:
        branch = filter_text(branch_reg.search(course)[1])
    except TypeError:
        branch = "None"
    if('(' in course):
        course = course[0:course.find('(')].strip()
    info = {
        'college' : college,
        'semester' : semester,
        'course' : course,
        'branch' : branch,
    }
    return info

In [11]:
# def add_to_dict(info,fulldic) :
#     array=['college','course','branch','semester']
#     tempdic=fulldic
#     for i in range(len(array)-1):
#         key=array[i]
#         nextkey=array[i+1]
#         if(tempdic.get(info[key])==None):
#             tempdic[info[key]]={}
#         tempdic[info[key]][info[nextkey]]={}
#         tempdic=tempdic[info[key]]

# def get_sem(info,fulldic):
#     array=['college','course','branch','semester']
#     tempdic=fulldic
#     for i in array:
#         if tempdic.get(i)==None:
#             return False
#         else:
#             tempdic=tempdic.get(i)
#     return True

In [12]:
SITE = "https://api-rhapsody.herokuapp.com/academia"
# SITE = "http://localhost:3000/academia"

In [13]:
# #Add college
# data ={ 
#     'college' : {
#         'college' : "University School Of Information, Communication & Technology (Formerly Usit)"
#         }
# }
# r = requests.post(SITE+"/college",json=data)
# print(r,r.content)


In [14]:
def already_exists(info) :
    r = requests.get(SITE+"/semester",params=info)
    content = json.loads(r.content)
#     print(r.status_code,r.content)
    return r.status_code == 200 and content != {}
    

In [15]:
def getSubjects(df) :
    subjects = []
    for index,row in df.iterrows() :
        subject = {}
        subject['subject'] = row['Subject'].strip().title()
        subject['subjectCode'] = row['Code']
        subject['credits'] = row['Credit']
        subjects.append(subject)
    return subjects

In [16]:
def pdfextract(filename,fulldic):
    sno=fulldic.get()
    success=True
    pdf = PyPDF2.PdfFileReader(filename)
#     print(filename,pdf.getNumPages())
    pages=pdf.getNumPages()
    for i in range(0,pages) :
        text = pdf.getPage(i).extractText()
        text=text.replace("\n",'')
        if scheme_reg.search(text) :
            try:
                info = get_info(text)
                df = tabula.read_pdf(filename,pages=i+1)
                subjects = getSubjects(df[0])     
                print(f"{sno}:{info}")
            except Exception as e:
                success=False
                print(f"{filename}:{i}")
                print(e)
                continue
            
            if already_exists(info) :
#                 print("information already exists")
                continue
            info['semester'] = {'semester' : info['semester'], 'subjects' : subjects}
            data ={ 
                'college' : {
                    'college' : info["college"]
                    }
            }
            # if(fulldic.get(info['college'])==None):
            r = requests.post(SITE+"/college",json=data)
            r = requests.post(SITE+"/semester",json=info)
            if r.status_code!=200:
                print(f"{filename}:{i}")
                print(r,r.content)
                print(info)
    return success

In [17]:
def download(_):
    url,filename,fulldic = _
    filename=str(filename)+".pdf"
    r=requests.get(url,allow_redirects=True)
    f=open(filename,'wb')
    f.write(r.content)
    f.close()
    _=pdfextract(filename,fulldic)):    
    if(_==True):
        os.remove(filename)
    return info

In [20]:
from itertools import repeat
import pandas as pd
start=0
end=len(ahrefs)
with Manager() as manager:
    fulldic = manager.Queue()
    for i in range(start,end,1):
        fulldic.put(i)
    with Pool(8) as pool:
        for _ in pool.imap_unordered(download,list(zip(ahrefs[start:end],range(start,end,1),repeat(fulldic,end-start)))):
          pass

1:{'college': 'Ambedkar Institute Of Advanced Communication Technologies And Research (Formerly Ait)', 'semester': 4, 'course': 'Master Of Technology', 'branch': 'Rf And Microwave Engineering'}
0:{'college': 'Ambedkar Institute Of Advanced Communication Technologies And Research (Formerly Ait)', 'semester': 4, 'course': 'Master Of Technology', 'branch': 'Signal Processing'}
2:{'college': 'Ambedkar Institute Of Advanced Communication Technologies And Research (Formerly Ait)', 'semester': 4, 'course': 'Master Of Technology', 'branch': 'Digital Communication'}
3:{'college': 'University School Of Humanities And Social Sciences', 'semester': 4, 'course': 'Master Of Arts', 'branch': 'Economics'}
5:{'college': 'University School Of Humanities And Social Sciences', 'semester': 4, 'course': 'Master Of Arts', 'branch': 'English'}
6:{'college': 'University School Of Information, Communication And Technology (Formerly Usit)', 'semester': 8, 'course': 'Bachelor Of Technology', 'branch': 'Computer S

Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-7:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/p

FileNotFoundError: ignored

In [ ]:
import pprint
pprint.pprint(fulldic)

In [ ]:
from IPython.display import display